# Inicialização do ambiente

In [ ]:
# Essentials
import pandas as pd
import numpy as np

import datetime as dt
from datetime import timedelta
import gc, math, warnings, time
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# Plots
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Stats
from scipy import stats
import ppscore as pps
import statsmodels.api as sm
from scipy.stats import skew, norm

# Estebelece limites para visualização no notebook
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',500)

# Limita a 3 casas decimais a apresentação das variaveis tipo float
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

# Carga de dados do arquivo CSV básico e análise

In [ ]:
# Carrega dados do arquivo
df_blk = pd.read_csv('../temp/vmblacklist.csv', sep=';')  # Lista de máquinas que não devem ser ajustadas
df = pd.read_csv('../temp/vm_list.csv', sep=';', parse_dates=['timestamp'])  # Lista total de VMs

In [ ]:
# Define periodo de análise em dias e remove medições do dia de hoje
periodo = 7

#Define variaveis e datasets cmo preparação para execução da rotina de análise
cols = ['vmname', 'weekday', 'minimo', 'maximo', 'media', '50%', '95%']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekDays = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5: "Saturday", 6:"Sunday"}
ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fileout= '../fig/{}.jpg'
    
# Cria dataframe para armazenamento de dados estatísticos
df_data = pd.DataFrame(columns=cols)

In [ ]:
# Remove máquinas da black list da amostra
dfx = df[df['vmname'].isin(df_blk['vmname']) == False]

# Filtra amostra de dados para o periodo de 7 dias contados a partir do dia anterior à ultima data armazenada.
# Isso garante dias completos
end_date = dfx['timestamp'].max()

# Calcula data final da amostra - 1 dia antes da data da últime medição
end_date = end_date - timedelta(1)

# Ajusta horário para final do dia
end_date = end_date.strftime('%Y-%m-%d') + ' 23:59:59'

# Converte novamente para o formato datetime
end_date = pd.to_datetime(end_date)

# Calcula data inicial da amostra = 7 dias antes da data final
initial_date = (pd.to_datetime(end_date.strftime('%Y-%m-%d') + ' 00:00:00')) - timedelta(periodo-1)

dfx = dfx[(dfx['timestamp'] > initial_date) & (dfx['timestamp'] <= end_date)]


# Garante que o campo timestamp está no formato de datetime
dfx['timestamp'] = pd.to_datetime(dfx['timestamp']) 

# Ordena registros
dfx.sort_values(by=['timestamp'], inplace=True)

vm_list = list(dfx['vmname'].unique())

# Seta definições de tamanho para apresentação de gráficos
f, ax = plt.subplots(nrows=3, ncols=4)
plt.rcParams['figure.figsize'] = [17, 18]
plt.subplots_adjust(wspace=0.15, hspace=0.17)
plt.close('all')

In [ ]:
for vms in vm_list:
    # ETAPA 1 ===========================================
    # Filtra base de dados para conter somente registros relacionados às VMs selecionadas
    df_aux1 = dfx[dfx['vmname'] == vms]

    # Reorganiza indices
    df_aux1.reset_index(drop=True, inplace=True)

    # Define coluna timestamp como indice
    df_aux1.set_index(['timestamp'], inplace=True)
    
    # ETAPA 2 ===========================================
    # Fazendo resample dos registros considerando consolidações a cada 1 minuto
    df_aux1 = df_aux1.resample('1min').mean()

    # Preenche valores faltantes utilizando KNN
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=3)
    df_aux2 = imputer.fit_transform(df_aux1)
    # Remontando dataframe
    df_aux1 = pd.DataFrame(df_aux2, index=df_aux1.index, columns=df_aux1.columns)
    del df_aux2
    gc.collect();
    
    # Adiciona novas colunas para gráficos e ajusta base para apresentação em determinados tipos de gráficos
    df_aux2 = df_aux1.copy()
    df_aux2['weekday'] = df_aux2.index.weekday
    df_aux2['weekday'] = df_aux2['weekday'].map(weekDays)
    
    # ETAPA 3 - Gráficos ================================
    f, ax = plt.subplots(nrows=3, ncols=4)
    plt.rcParams['figure.figsize'] = [17, 18]
    plt.subplots_adjust(wspace=0.15, hspace=0.17)

    ax1 = plt.subplot2grid((3, 4), (0, 0), colspan=4, rowspan=1)
    ax2 = plt.subplot2grid((3, 4), (1, 0))
    ax3 = plt.subplot2grid((3, 4), (1, 1))
    ax4 = plt.subplot2grid((3, 4), (1, 2), colspan=2, rowspan=1)
    ax5 = plt.subplot2grid((3, 4), (2, 0), colspan=4, rowspan=1)

    # Gráfico 1 - Linha
    sns.lineplot(data = df_aux1, x=df_aux1.index, y=df_aux1.cpu, ax=ax1)
    ax1.grid(color='lightgray', linestyle='-', linewidth=0.2)
    ax1.set_title(vms + ' - Período de 7 dias: ' + initial_date.strftime('%d/%m/%Y %H:%M') + \
                  ' a ' + end_date.strftime('%d/%m/%Y %H:%M'), fontsize=14)
    ax1.set_xlabel('')
    ax1.set_ylabel('% de consumo de CPU', fontsize=12)

    # Gráfico 2 - Histograma básico
    df_aux1['cpu'].hist(bins = 30, ax = ax2)
    ax2.set_title('% de consumo de CPU', fontsize=12)
    
    # Gráfico 3- Histograma com curva normal
    #if df_aux1['cpu'].max() != 0:
    if len(df_aux1['cpu'].unique()) > 1:
        sns.distplot(df_aux1['cpu'], ax = ax3, fit=norm)
        ax3.set_title('% de consumo de CPU', fontsize=12)
        ax3.set_xlabel('')
    
    # Gráfico 4 - Boxplot
    sns.boxplot(x='weekday', y='cpu', data=df_aux2, ax = ax4, orient="v", order=ordem, palette="Set2")
    ax4.set_title('% de consumo de CPU por dia da semana', fontsize=12)
    ax4.set_xlabel('')
    ax4.set_ylabel('')
    
    
    # ETAPA 4 - Métricas estatisticas ===================
    # Popula dataframe auxiliar com dados estatísticos das metricas dos dias da semana
    df_aux3 = df_aux2.groupby(['weekday']).describe(percentiles=[0.95]).reset_index().round(decimals=3)
        
    # remove colunas indesejadas e estrutura multiindex e renomeia colunas
    df_aux3.drop('count', axis=1, level= 1, inplace=True)
    df_aux3.columns = [''.join(col).strip() for col in df_aux3.columns.values]
    df_aux3.rename(columns = {'cpumean':'media', 'cpustd': 'desvio', 'cpumin': 'minimo', 
                             'cpumax': 'maximo', 'cpu50%': '50%', 'cpu95%': '95%'}, inplace = True)
    df_aux3['vmname'] = vms
   
    #Cria coluna adicional para 3sigma (linha de corte para análise de redução)
    df_aux3['3sigma'] = 3 * df_aux3['desvio'] + df_aux3['media']
        
    # Cria dataframe auxiliar com indicadores estatísticos de todo o periodo
    df_aux1 = df_aux2.describe(percentiles=[0.95]).reset_index().round(decimals=3)
    df_aux1 = df_aux1.T.reset_index()
    header_row = 0
    df_aux1.columns = df_aux1.iloc[header_row]
    
    # Remove colunas e linhas desnecessárias e renomeia colunas
    df_aux1.drop(['index', 'count'], axis=1, inplace=True)
    df_aux1.drop(0, axis=0, inplace=True)
    df_aux1.rename(columns = {'mean':'media', 'std': 'desvio', 'min': 'minimo', 
                              'max': 'maximo', '50%': '50%', '95%': '95%'}, inplace = True)
    df_aux1['vmname'] = vms
    df_aux1['weekday']= 'AllDays'
    
    #Cria coluna adicional para 3sigma (linha de corte para análise de redução)
    df_aux1['3sigma'] = 3 * df_aux1['desvio'] + df_aux1['media']

    # Acrescenta dados da metrica na base de estatísticas
    df_aux2 = pd.concat([df_aux1, df_aux3], ignore_index=True)
    df_aux2.drop(['vmname'], axis=1, inplace=True)
    df_aux2 = df_aux2[['weekday', 'minimo', 'maximo', 'media', 'desvio', '50%', '95%', '3sigma']]
    
    # Converte variaveis numéricas para formato float e arredonda em 3 casas decimais
    for col in df_aux2.columns[1:]:
        df_aux2[col] = df_aux2[col].astype('float')
        df_aux2[col] = df_aux2[col].round(decimals=3)
    
    df_aux2['3sigma'] = df_aux2['3sigma'].round(decimals=0)
    df_data = pd.concat([df_data, df_aux1, df_aux3], ignore_index=True)
       

    # Grafico 5 - Tabela com dados
    ax5.xaxis.set_visible(False)  
    ax5.yaxis.set_visible(False)  
    ax5.set_frame_on(False)  
    tabla = table(ax5, df_aux2, loc='upper right', colWidths=[0.07]*len(df_aux2.columns))  
    tabla.auto_set_font_size(False) 
    tabla.set_fontsize(12) 
    tabla.scale(1.7, 1.2) 
    
    # Salva gráfico em arquivo jpg
    plt.savefig(fileout.format(vms), format='jpg')

    plt.show();
    

# Ajustando colunas da base e arredondando resultados numericos 
df_data = df_data[['vmname', 'weekday', 'minimo', 'maximo', 'media', 'desvio', '50%', '95%', '3sigma']]
for col in df_data.columns[2:]:
    df_data[col] = df_data[col].astype('float')
    df_data[col] = df_data[col].round(decimals=3)

df_data['3sigma'] = df_data['3sigma'].round(decimals=0)

# Gravando arquivo final
df_data.to_csv('../temp/estatisticas.csv', sep=';', decimal=',', encoding='utf-8', index=False)

# Adicionando informação à tabela de estatisticas

In [ ]:
# Carrega dados dos arquivos
# Carrega arquivo csv com lista de tamanhos, renomeia coluna e seleciona as que interessam para montar dataframe
df_vms = pd.read_csv('../temp/geral.csv', sep=';')              # Lista de VMs 
df_est = pd.read_csv('../temp/estatisticas.csv', sep=';', decimal=',')   # Estatísticas das VMs
cols = ['vmsize', 'cpu', 'ram_gb', 'serie', 'order']
df_tam = pd.read_csv('../temp/tamanhos.csv', sep=';', header=0, names=cols, usecols=cols)  # Catálogo de series e tamanhos

In [ ]:
# Mantem em df_est somente registros do consolidado da semana (AllDays)
df_est.drop(df_est[df_est['weekday']!='AllDays'].index, axis=0, inplace=True)

# Mantem em df_vms somente registros de servidores que existam na base de estatisticas
df_vms = df_vms[df_vms['vmname'].isin(df_est['vmname']) == True]

# Ajusta tamanhos de ram para escala em GB
df_tam['ram_gb'] /= 1024
df_tam['ram_gb'].round(2)

# Ordena df_tam por vmsize e cpu
df_tam.sort_values(by=['serie', 'order'], inplace=True, ignore_index=True)

In [ ]:
# Faz merge das tabelas df_est e df_vms
df_est = df_est.merge(df_vms, how='right', left_on='vmname', right_on='vmname')

# Faz merge das tabelas df_est e df_tam
df_est = df_est.merge(df_tam, how='left', left_on='vmsize', right_on='vmsize')

### Remove da lista servidores com 1 cpu e com 3sigma maior que 40% ou 3sigma igual a zero

In [ ]:
df_est.drop(df_est[df_est['cpu'] == 1.0].index, axis = 0, inplace=True)
df_est.drop(df_est[(df_est['3sigma'] > 35.0) | (df_est['3sigma'] == 0.0)].index, axis = 0, inplace=True)

# Ordena lista por vmsize, cpu, ram e nome
df_est.sort_values(by=['3sigma', 'serie', 'order'], inplace=True, ignore_index=True)

### Captura métricas atuais e analisa opções

In [ ]:
# Prepara df_est para novos dados
cols = ['new_vmsize', 'new_cpu', 'new_ram_gb', 'new_3sigma']
df_est[cols] = ['', 0.0, 0.0, 0.0]

# Indexa tabela df_est pelo vmname
df_est.set_index(['vmname'], inplace=True)

# Ordena base de tamanhos de forma decrescente
df_tam.sort_values(by=['serie', 'order', 'vmsize'], ascending=False, ignore_index=True, inplace=True)

In [ ]:
for vms in df_est.index:
    # Seleciona dados atuais do servidor em análise
    cpu_atual = df_est.loc[vms, 'cpu']
    ram_atual = df_est.loc[vms, 'ram_gb']
    serie_atual = df_est.loc[vms, 'serie']
    order_atual = df_est.loc[vms, 'order']
    sigma_atual = df_est.loc[vms, '3sigma']

    cpu_new = 0
    sigma_new = 0
    vmsize_new = ''
    
    # Separa em df_tam conjunto de tamanhos da mesma serie mas com número de ordem inferior ao atual
    df = df_tam[(df_tam['serie']==serie_atual) & (df_tam['order']<=(order_atual-1))].reset_index(drop=True)
    
    # Analisa informações de vms e busca por servidor com tamanho menor
    for key, value in enumerate(df['vmsize']):
        cpu = df.loc[key, 'cpu']
        ram = df.loc[key, 'ram_gb']
        if (cpu < cpu_atual) and (ram >= ram_atual/2):
            cpu_new = cpu
            vmsize_new = df.loc[key, 'vmsize']
            sigma_new = (cpu_atual * sigma_atual / cpu_new) * 1.10
            break

    if cpu_new != 0:
        # Complementa informações em tabela de estatísticas df_est
        df_est.loc[vms, 'new_vmsize'] = vmsize_new
        df_est.loc[vms, 'new_cpu'] = cpu_new
        df_est.loc[vms, 'new_ram_gb'] = ram
        df_est.loc[vms, 'new_3sigma'] = sigma_new
    else:
        # Não foi encontrada nenhuma máquina dentro da mesma serie que atenda aos requisitos de redução
        df_est.loc[vms, 'new_vmsize'] = 'NA'
        df_est.loc[vms, 'new_cpu'] = 0
        df_est.loc[vms, 'new_ram_gb'] = 0
        df_est.loc[vms, 'new_3sigma'] = 0

# Ajustando valores e apresentação final
df = df_est.drop(df_est[df_est['new_vmsize']=='NA'].index, axis=0)
df.reset_index(inplace=True)
df.sort_values(by=['vmname'], ignore_index=True, inplace=True)

df.round(decimals=3)
df['new_cpu'] = df['new_cpu'].astype('int32')
df['new_3sigma'] = df['new_3sigma'].round(decimals=0)
df.drop(['order'], axis=1, inplace=True)

In [ ]:
# Grava arquivo de recomendações 
df.to_excel('../data/recomenda.xlsx', float_format="%7.3f", sheet_name='Recomendações', index=False)